<a href="https://colab.research.google.com/github/Ao-27/Home_Sales_Mod22_BigData/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'

#export PYTHONPATH=$SPARK_HOME/python/:$PYTHONPATH #this didn't work ... but I can't remember the url it came from.
#export PYTHONPATH=$SPARK_HOME/python/lib/py4j-0.8.2.1-src.zip:$PYTHONPATH

spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version


  #fixed below via url => https://stackoverflow.com/questions/55240940/error-while-installing-spark-on-google-colab
  #Needed to point to the archive url as noted on stackoverflow ...

# Install Spark and Java
!apt-get update

# innstall java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf $SPARK_VERSION-bin-hadoop3.tgz #3.2 ... possibly ...

# set your spark folder to your system path environment.
  # Set Environment Variables

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"


# install findspark using pip
!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [46]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [47]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [48]:
#Create DF via url access.

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, header=True)
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [49]:
# 2. Create a temporary view of the DataFrame.

Temp_view1 = home_sales_df.createOrReplaceTempView("home_sales")
Temp_view1

In [50]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query1 = spark.sql("""
SELECT YEAR(date),
ROUND(AVG(price),2) AS avg_4bdrm_price

FROM
home_sales

WHERE
bedrooms = 4

GROUP BY YEAR(date);""").show()

query1

+----------+---------------+
|year(date)|avg_4bdrm_price|
+----------+---------------+
|      2022|      296363.88|
|      2019|       300263.7|
|      2020|      298353.78|
|      2021|      301819.44|
+----------+---------------+



In [51]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query2 = spark.sql("""
SELECT date_built,
ROUND(AVG(price),2) AS avg_3bdrm3bath_price

FROM
home_sales

WHERE
bedrooms = 3
AND
bathrooms = 3

GROUP BY date_built;""").show()

query2


+----------+--------------------+
|date_built|avg_3bdrm3bath_price|
+----------+--------------------+
|      2015|            288770.3|
|      2013|           295962.27|
|      2014|           290852.27|
|      2012|           293683.19|
|      2016|           290555.07|
|      2010|           292859.62|
|      2011|           291117.47|
|      2017|           292676.79|
+----------+--------------------+



In [52]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query3 = spark.sql("""
SELECT date_built,
ROUND(AVG(price),2) AS avg_homeprice_2000sqft

FROM
home_sales

WHERE
bedrooms = 3
AND
bathrooms = 3
AND
floors = 2
AND
sqft_living >= 2000

GROUP BY date_built;""").show()

query3

+----------+----------------------+
|date_built|avg_homeprice_2000sqft|
+----------+----------------------+
|      2015|             297609.97|
|      2013|             303676.79|
|      2014|             298264.72|
|      2012|             307539.97|
|      2016|              293965.1|
|      2010|             285010.22|
|      2011|             276553.81|
|      2017|             280317.58|
+----------+----------------------+



In [53]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query4 = spark.sql("""
SELECT view,
ROUND(AVG(price),2) AS avg_homeprice_above_350000

FROM
home_sales

GROUP BY
view

HAVING
ROUND(AVG(price),2) >= 350000;""").show()

timer1 = print("--- %s seconds ---" % (time.time() - start_time)) #took 0.6655 seconds ...

timer1
query4

+----+--------------------------+
|view|avg_homeprice_above_350000|
+----+--------------------------+
|  85|                1056336.74|
|  65|                 736679.93|
|  53|                  755214.8|
|  78|                1080649.37|
|  81|                1053472.79|
|  76|                1058802.78|
|  91|                1137372.73|
|  93|                1026006.06|
|  52|                 733780.26|
|  86|                1070444.25|
|  94|                 1033536.2|
|  57|                  734340.5|
|  54|                 798684.82|
|  96|                1017815.92|
|  92|                 970402.55|
|  64|                 767036.67|
|  61|                 746877.59|
|  88|                1031719.35|
|  72|                 780914.67|
|  59|                  791453.0|
+----+--------------------------+
only showing top 20 rows

--- 0.6655106544494629 seconds ---


In [54]:
# 7. Cache the the temporary table home_sales.
cache_temp_view = spark.sql("cache table home_sales")
cache_temp_view

DataFrame[]

In [55]:
# 8. Check if the table is cached.

cache_sanchk = spark.catalog.isCached('home_sales')

cache_sanchk

True

In [56]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime. => The cached data performed the query significantly faster. 0.6655 / 0.2178 = 3.05 ... it was 3 times faster.

start_time = time.time()

query5_c = spark.sql("""
SELECT view,
ROUND(AVG(price),2) AS avg_homeprice_above_350000

FROM
home_sales

GROUP BY
view

HAVING
ROUND(AVG(price),2) >= 350000;""").show()

timer2 = print("--- %s seconds ---" % (time.time() - start_time)) #took 0.2178 seconds ... The cached data performed the query significantly faster. 0.6655 / 0.2178 = 3.05 ... it was 3 times faster.

timer2
query5_c

+----+--------------------------+
|view|avg_homeprice_above_350000|
+----+--------------------------+
|  85|                1056336.74|
|  65|                 736679.93|
|  53|                  755214.8|
|  78|                1080649.37|
|  81|                1053472.79|
|  76|                1058802.78|
|  91|                1137372.73|
|  93|                1026006.06|
|  52|                 733780.26|
|  86|                1070444.25|
|  94|                 1033536.2|
|  57|                  734340.5|
|  54|                 798684.82|
|  96|                1017815.92|
|  92|                 970402.55|
|  64|                 767036.67|
|  61|                 746877.59|
|  88|                1031719.35|
|  72|                 780914.67|
|  59|                  791453.0|
+----+--------------------------+
only showing top 20 rows

--- 0.2178046703338623 seconds ---


In [ ]:
#My answer isn't legible in GitHub.

# Determine the runtime and compare it to the cached version.
# => The cached data performed the query significantly faster. 0.6655 / 0.2178 = 3.05 ... it was 3 times faster which is significant depending on the size of the dataset.

In [57]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

parquet1 = home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

parquet1


In [58]:
# 11. Read the parquet formatted data.
parquet1_df = spark.read.parquet('home_sales_partitioned')
parquet1_df

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [59]:
# 12. Create a temporary table for the parquet data.
parquet_temp_view = parquet1_df.createOrReplaceTempView('home_sales_parq')
parquet_temp_view

In [62]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places. # Determine the runtime and compare it to the cached version. => The Cached data was faster but not as significantly as the non-cached vs cached. Speed factor advantage comparison for parquet time = 0.3552 / 0.2178 = 1.63 times faster for the Cached.

start_time = time.time()

query6_p = spark.sql("""
SELECT view,
ROUND(AVG(price),2) AS avg_homeprice_above_350000

FROM
home_sales_parq

GROUP BY
view

HAVING
ROUND(AVG(price),2) >= 350000;""").show()


timer3 = print("--- %s seconds ---" % (time.time() - start_time))

timer3   #   0.3552 seconds as Parquet 1st run. Cached was faster at 0.2178. The Cached data was faster but not as significantly as the non-cached vs cached. Speed factor advantage comparison for parquet time = 0.3552 / 0.2178 = 1.63 times faster for the Cached.
query6_p

+----+--------------------------+
|view|avg_homeprice_above_350000|
+----+--------------------------+
|  85|                1056336.74|
|  65|                 736679.93|
|  53|                  755214.8|
|  78|                1080649.37|
|  81|                1053472.79|
|  76|                1058802.78|
|  91|                1137372.73|
|  93|                1026006.06|
|  52|                 733780.26|
|  86|                1070444.25|
|  94|                 1033536.2|
|  57|                  734340.5|
|  54|                 798684.82|
|  96|                1017815.92|
|  92|                 970402.55|
|  64|                 767036.67|
|  61|                 746877.59|
|  88|                1031719.35|
|  72|                 780914.67|
|  55|                 771153.32|
+----+--------------------------+
only showing top 20 rows

--- 0.6140885353088379 seconds ---


In [ ]:
#My answer isn't legible in GitHub.

# Determine the runtime and compare it to the cached version.
# => The Cached data was faster but not as significantly as the non-cached vs cached.
# Speed factor advantage comparison for parquet time = 0.3552 / 0.2178 = 1.63 times faster for the Cached.



In [65]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [67]:
# 15. Check if the home_sales is no longer cached

cache_sanchk = spark.catalog.isCached('home_sales')

if cache_sanchk: #=True defaulted ...
  print ("Not Cleared - Data is still cached for a Table")
else:
  print("Cleared - No Data is still cached.")

Cleared - No Data is still cached.
